Import necessary packages and create a bucket for Sagemaker

In [ ]:
import os
import io
import re
import time
import json
import boto3
import sagemaker
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sagemaker.amazon.common as smac

role = sagemaker.get_execution_role()
region = boto3.Session().region_name

bucket = sagemaker.Session().default_bucket()

prefix = (
    "sagemaker/mnist-prediction"  
)
print(bucket)

sagemaker-us-east-2-132656972280


Load the train data exported from DynamoDB into the S3 bucket to the sagemaker notebook instance

In [ ]:
s3 = boto3.client("s3")

filename = "mnist-train.csv"
s3.download_file(bucket, "mnist-train.csv", filename)
train_data = pd.read_csv(filename)
train_data = train_data.drop(columns = 'uuid', axis = 0)
print(train_data.shape)

display(train_data.head())
display(train_data.describe())
display(train_data.label.value_counts())

(60000, 785)


,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
count,60000.000000,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,...,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.0000,60000.0,60000.0,60000.0,60000.0
mean,4.453933,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.200433,0.088867,0.045633,0.019283,0.015117,0.0020,0.0,0.0,0.0,0.0
std,2.889270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.042472,3.956189,2.839845,1.686770,1.678283,0.3466,0.0,0.0,0.0,0.0
min,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
25%,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
50%,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
75%,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
max,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254.000000,254.000000,253.000000,253.000000,254.000000,62.0000,0.0,0.0,0.0,0.0


1    6742
7    6265
3    6131
2    5958
9    5949
0    5923
6    5918
8    5851
4    5842
5    5421
Name: label, dtype: int64

Load the test data exported from DynamoDB into the S3 bucket to the sagemaker notebook instance

In [ ]:
s3 = boto3.client("s3")

filename = "mnist-test.csv"
s3.download_file(bucket, "mnist-test.csv", filename)
test_data = pd.read_csv(filename)
test_data = test_data.drop(columns ='uuid', axis = 0)
print(test_data.shape)

display(test_data.head())
display(test_data.describe())
display(test_data.label.value_counts())

(10000, 785)


,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
count,10000.000000,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,...,10000.000000,10000.000000,10000.000000,10000.0000,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0
mean,4.443400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.179300,0.163600,0.052600,0.0006,0.0,0.0,0.0,0.0,0.0,0.0
std,2.895865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.674149,5.736072,2.420004,0.0600,0.0,0.0,0.0,0.0,0.0,0.0
min,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0
25%,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0
50%,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0
75%,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0
max,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,253.000000,253.000000,156.000000,6.0000,0.0,0.0,0.0,0.0,0.0,0.0


1    1135
2    1032
7    1028
3    1010
9    1009
4     982
0     980
8     974
6     958
5     892
Name: label, dtype: int64

Split train data into Training and Validation

In [ ]:
rand_split = np.random.rand(len(train_data))

train_list = rand_split < 0.9
val_list = rand_split >= 0.9

data_train = train_data[train_list]
data_val = train_data[val_list]
data_test = test_data

train_y = data_train.iloc[:, 0].to_numpy()
train_X = data_train.iloc[:, 1:].to_numpy()

val_y = data_val.iloc[:, 0].to_numpy()
val_X = data_val.iloc[:, 1:].to_numpy()

test_y = data_test.iloc[:, 0].to_numpy()
test_X = data_test.iloc[:, 1:].to_numpy()

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[7 2 1 ... 4 5 6]
[5 0 4 ... 5 6 8]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
train_file = "transfer_train.data"

f = io.BytesIO()
smac.write_numpy_to_dense_tensor(f, train_X.astype("float32"), train_y.astype("float32"))
f.seek(0)

boto3.Session().resource("s3").Bucket(bucket).Object(
    os.path.join(prefix, "train", train_file)
).upload_fileobj(f)

In [ ]:
validation_file = "transfer_validation.data"

f = io.BytesIO()
smac.write_numpy_to_dense_tensor(f, val_X.astype("float32"), val_y.astype("float32"))
f.seek(0)

boto3.Session().resource("s3").Bucket(bucket).Object(
    os.path.join(prefix, "validation", validation_file)
).upload_fileobj(f)

Create sagemaker training Framework

In [ ]:
from sagemaker import image_uris

container = image_uris.retrieve(region=boto3.Session().region_name, framework="linear-learner")

Create a Training Job for the model

In [ ]:
transfer_job = "Transfer-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

print("Job name is:", transfer_job)

transfer_training_params = {
    "RoleArn": role,
    "TrainingJobName": transfer_job,
    "AlgorithmSpecification": {"TrainingImage": container, "TrainingInputMode": "File"},
    "ResourceConfig": {"InstanceCount": 1, "InstanceType": "ml.c4.2xlarge", "VolumeSizeInGB": 10},
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://{}/{}/train/".format(bucket, prefix),
                    "S3DataDistributionType": "ShardedByS3Key",
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None",
        },
        {
            "ChannelName": "validation",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://{}/{}/validation/".format(bucket, prefix),
                    "S3DataDistributionType": "FullyReplicated",
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None",
        },
    ],
    "OutputDataConfig": {"S3OutputPath": "s3://{}/{}/".format(bucket, prefix)},
    "HyperParameters": {
        "feature_dim": "784",
        "mini_batch_size": "100",
        "predictor_type": "regressor",
        "epochs": "50",
        "num_models": "32",
        "loss": "absolute_loss",
        },
    "StoppingCondition": {"MaxRuntimeInSeconds": 60 * 60},
}

Job name is: Transfer-2022-11-13-04-50-31


In [ ]:
%%time

region = boto3.Session().region_name
sm = boto3.client("sagemaker")

sm.create_training_job(**transfer_training_params)

status = sm.describe_training_job(TrainingJobName=transfer_job)["TrainingJobStatus"]
print(status)
sm.get_waiter("training_job_completed_or_stopped").wait(TrainingJobName=transfer_job)
if status == "Failed":
    message = sm.describe_training_job(TrainingJobName=transfer_job)["FailureReason"]
    print("Training failed with the following error: {}".format(message))
    raise Exception("Training job failed")

InProgress
CPU times: user 86.9 ms, sys: 736 µs, total: 87.7 ms
Wall time: 6min


In [ ]:
transfer_hosting_container = {
    "Image": container,
    "ModelDataUrl": sm.describe_training_job(TrainingJobName=transfer_job)["ModelArtifacts"][
        "S3ModelArtifacts"
    ],
}

create_model_response = sm.create_model(
    ModelName=transfer_job, ExecutionRoleArn=role, PrimaryContainer=transfer_hosting_container
)

print(create_model_response["ModelArn"])

arn:aws:sagemaker:us-east-2:132656972280:model/transfer-2022-11-13-04-50-31


Configure Endpoint for Sagemaker

In [ ]:
transfer_endpoint_config = "Transfer-endpoint-config-" + time.strftime(
    "%Y-%m-%d-%H-%M-%S", time.gmtime()
)
print(transfer_endpoint_config)
create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=transfer_endpoint_config,
    ProductionVariants=[
        {
            "InstanceType": "ml.p2.xlarge",
            "InitialInstanceCount": 1,
            "ModelName": transfer_job,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

Transfer-endpoint-config-2022-11-13-04-57-03
Endpoint Config Arn: arn:aws:sagemaker:us-east-2:132656972280:endpoint-config/transfer-endpoint-config-2022-11-13-04-57-03


Create endpoint for sagemaker

In [ ]:
%%time

transfer_endpoint = "Transfer-endpoint-" + time.strftime("%Y%m%d%H%M", time.gmtime())
print(transfer_endpoint)
create_endpoint_response = sm.create_endpoint(
    EndpointName=transfer_endpoint, EndpointConfigName=transfer_endpoint_config
)
print(create_endpoint_response["EndpointArn"])

resp = sm.describe_endpoint(EndpointName=transfer_endpoint)
status = resp["EndpointStatus"]
print("Status: " + status)

sm.get_waiter("endpoint_in_service").wait(EndpointName=transfer_endpoint)

resp = sm.describe_endpoint(EndpointName=transfer_endpoint)
status = resp["EndpointStatus"]
print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

if status != "InService":
    raise Exception("Endpoint creation failed")

Transfer-endpoint-202211130457
arn:aws:sagemaker:us-east-2:132656972280:endpoint/transfer-endpoint-202211130457
Status: Creating
Arn: arn:aws:sagemaker:us-east-2:132656972280:endpoint/transfer-endpoint-202211130457
Status: InService
CPU times: user 104 ms, sys: 7.63 ms, total: 112 ms
Wall time: 4min 1s


Function to convert numpy array to csv before sending it to the model for prediction

In [ ]:
def np2csv(arr):
    csv = io.BytesIO()
    np.savetxt(csv, arr, delimiter=",", fmt="%g")
    return csv.getvalue().decode().rstrip()

Predicting the label for the test data model for the first 1000 rows

In [ ]:
runtime = boto3.client("runtime.sagemaker")

payload = np2csv(test_X[:1000, :])
response = runtime.invoke_endpoint(
    EndpointName=transfer_endpoint, ContentType="text/csv", Body=payload
)
result = json.loads(response["Body"].read().decode())
test_pred = np.array([r["score"] for r in result["predictions"]])

In [ ]:
test_mae_transfer = np.mean(np.abs(test_y[:1000] - test_pred))
test_pred_class = (test_pred > 0.5) + 0
prediction_accuracy = np.mean((test_y == test_pred_class)) * 100

Print the error and Accuracy

In [6]:
print("Test Mean Absolute Error Transfer:", round(test_mae_transfer, 3))
print("Prediction Accuracy:", round(prediction_accuracy, 1), "%")

Test Mean Absolute Error Transfer: 1.411
Prediction Accuracy: 18.5 %
